# Tuning du modèle

L'objet de ce notebook est d'illustrer les différentes étapes de tuning du modèle.


## Préambule

### Imports

In [1]:
# setting up sys.path for relative imports
from pathlib import Path
import sys
project_root = str(Path(sys.path[0]).parents[1].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)

In [30]:
# imports and customization of diplay
# import os
import re
from functools import partial
from itertools import product
# import numpy as np
import pandas as pd
pd.options.display.min_rows = 6
pd.options.display.width=108
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
# from matplotlib import pyplot as plt

from src.pimest import ContentGetter
from src.pimest import PathGetter
from src.pimest import PDFContentParser
from src.pimest import BlockSplitter
from src.pimest import SimilaritySelector
# from src.pimest import custom_accuracy
from src.pimest import text_sim_score
# from src.pimest import text_similarity
# from src.pimest import build_text_processor

### Acquisition des données

On récupère les données manuellement étiquetées et on les intègre dans un dataframe

In [3]:
ground_truth_df = pd.read_csv(Path('..') / '..' / 'ground_truth' / 'manually_labelled_ground_truth.csv',
                              sep=';',
                              encoding='latin-1',
                              index_col='uid')
ground_truth_uids = list(ground_truth_df.index)

acqui_pipe = Pipeline([('PathGetter', PathGetter(ground_truth_uids=ground_truth_uids,
                                                  train_set_path=Path('..') / '..' / 'ground_truth',
                                                  ground_truth_path=Path('..') / '..' / 'ground_truth',
                                                  )),
                        ('ContentGetter', ContentGetter(missing_file='to_nan')),
                        ('ContentParser', PDFContentParser(none_content='to_empty')),
                       ],
                       verbose=True)

texts_df = acqui_pipe.fit_transform(ground_truth_df)
texts_df['ingredients'] = texts_df['ingredients'].fillna('')
texts_df

[Pipeline] ........ (step 1 of 3) Processing PathGetter, total=   0.1s
[Pipeline] ..... (step 2 of 3) Processing ContentGetter, total=   0.1s
Launching 8 processes.
[Pipeline] ..... (step 3 of 3) Processing ContentParser, total=  36.8s


,designation,ingredients,path,content,text
uid,,,,,
a0492df6-9c76-4303-8813-65ec5ccbfa70,Concentré liquide Asian en bouteille 980 ml CHEF,"Eau, maltodextrine, sel, arômes, sucre, arôme ...",../../ground_truth/a0492df6-9c76-4303-8813-65e...,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,Concentré Liquide Asian CHEF® \n\nBouteille de...
d183e914-db2f-4e2f-863a-a3b2d054c0b8,Pain burger curry 80 g CREATIV BURGER,"Farine de blé T65, eau, levure, vinaigre de ci...",../../ground_truth/d183e914-db2f-4e2f-863a-a3b...,b'%PDF-1.5\r%\xe2\xe3\xcf\xd3\r\n4 0 obj\r<</L...,
ab48a1ed-7a3d-4686-bb6d-ab4f367cada8,Macaroni en sachet 500 g PANZANI,- 100% Semoule de BLE dur de qualité supérieur...,../../ground_truth/ab48a1ed-7a3d-4686-bb6d-ab4...,b'%PDF-1.4\n%\xc7\xec\x8f\xa2\n5 0 obj\n<</Len...,Direction Qualité \n\n \n\n \n\nPATES ALIMENTA...
...,...,...,...,...,...
e67341d8-350f-46f4-9154-4dbbb8035621,PRÉPARATION POUR CRÈME BRÛLÉE BIO 6L,"Sucre roux de canne*° (64%), amidon de maïs*, ...",../../ground_truth/e67341d8-350f-46f4-9154-4db...,b'%PDF-1.7\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,FICHE TECHNIQUE \n\nCREME BRÛLÉE 6L \n\nREF : ...
a8f6f672-20ac-4ff8-a8f2-3bc4306c8df3,Céréales instantanées en poudre saveur caramel...,"Farine 87,1 % (Blé (GLUTEN), Blé hydrolysé (GL...",../../ground_truth/a8f6f672-20ac-4ff8-a8f2-3bc...,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,81 rue de Sans Souci – CS13754 – 69576 Limones...
0faad739-ea8c-4f03-b62e-51ee592a0546,"FARINE DE BLÉ TYPE 45, 10KG",Farine de blé T45,../../ground_truth/0faad739-ea8c-4f03-b62e-51e...,b'%PDF-1.5\r\n%\xb5\xb5\xb5\xb5\r\n1 0 obj\r\n...,\n1050/10502066400 \n\n10502055300/1050202520...


### Train / Test split

On va appliquer une grid search pour déterminer les meilleurs paramètres de notre modèle. 
Pour ne pas surestimer la performance du modèle, il est nécessaire de bien séparer le jeu de test du jeu d'entraînement, y compris pour la grid search !

In [4]:
train, test = train_test_split(texts_df, test_size=100, random_state=42)

Dans toute la suite, on utilisera le jeu d'entraînement pour effectuer le tuning des hyperparamètres.

## Ajustement de la fonction de découpage des textes

L'objectif de cette partie est d'optimiser la fonction de découpage des textes en blocs. On va tester quelques fonctions candidates, via une GridSearch.

### Définition des fonctions candidates

On définit les fonctions de split : 

In [5]:
# definitions of splitter funcs
splitter_funcs = []
def split_func(text):
    return(text.split('\n\n'))
splitter_funcs.append(split_func)
def split_func(text):
    return(text.split('\n'))
splitter_funcs.append(split_func)
def split_func(text):
    regex = r'\s*\n\s*\n\s*'
    return(re.split(regex, text))
splitter_funcs.append(split_func)

### Mise en place du pipeline

On construit ensuite un pipeline de traitement du texte.
Le SimilaritySelector prenant en entrée une pandas.Series, on définit entre le BlockSplitter (dont la méthode transform() retourne un pandas.DataFrame) et le SimilaritySelector une fonction utilitaire qui séléctionne la colonne 'blocks'.

In [6]:
def select_col(df, col_name='blocks'):
        return(df[col_name].fillna(''))
col_selector = FunctionTransformer(select_col)    

In [23]:
process_pipe = Pipeline([('Splitter', BlockSplitter()),
                         ('ColumnSelector', col_selector),
                         ('SimilaritySelector', SimilaritySelector())
                        ],
                       verbose=False)

On peut tester le fonctionnement de ce Pipeline.
Attention, les résultats ne sont pas représentatifs, on entraîne et on prédit sur le même jeu de données !

In [8]:
process_pipe.fit(train, train['ingredients'])
process_pipe.predict(train)

Launching 8 processes.
Launching 8 processes.


uid
02d5ceb9-21c2-4965-8f65-309bca7638b2    Café chicorée solubles et fibres de chicorée.\...
bbe72396-6ed4-4df1-935b-0c0a7dbd77dc                                                     
507b428e-e99d-464b-b9d3-50629efe4355    COMPOSITION\nMélange de Blés de pays recommand...
                                                              ...                        
4b28bb17-1f1d-4cbb-ac3b-80227ef248ab    Gluten\nCrustacés\nOeufs\nPoisson\nSoja\nLait\...
d2137dae-ff21-46ec-83be-7400773c6c3b    Amidon modifié de pomme de terre - Fécule de p...
571d98ae-9647-4bd4-ad1a-a497f93987cb    Composition typique (Données inappropriées pou...
Length: 400, dtype: object

### Application de la GridSearch

On applique ensuite une grid search en faisant varier les fonctions de split.

In [24]:
process_pipe.get_params()

{'memory': None,
 'steps': [('Splitter', <src.pimest.BlockSplitter at 0x7f250c6b6cd0>),
  ('ColumnSelector',
   FunctionTransformer(func=<function select_col at 0x7f252a0d5dc0>)),
  ('SimilaritySelector', <src.pimest.SimilaritySelector at 0x7f250c6b6bb0>)],
 'verbose': False,
 'Splitter': <src.pimest.BlockSplitter at 0x7f250c6b6cd0>,
 'ColumnSelector': FunctionTransformer(func=<function select_col at 0x7f252a0d5dc0>),
 'SimilaritySelector': <src.pimest.SimilaritySelector at 0x7f250c6b6bb0>,
 'Splitter__source_col': 'text',
 'Splitter__target_col': 'blocks',
 'Splitter__target_exists': 'raise',
 'Splitter__splitter_func': <function src.pimest.BlockSplitter.<lambda>(x)>,
 'ColumnSelector__accept_sparse': False,
 'ColumnSelector__check_inverse': True,
 'ColumnSelector__func': <function __main__.select_col(df, col_name='blocks')>,
 'ColumnSelector__inv_kw_args': None,
 'ColumnSelector__inverse_func': None,
 'ColumnSelector__kw_args': None,
 'ColumnSelector__validate': False,
 'SimilaritySe

In [9]:
lev_scorer = partial(text_sim_score, similarity='levenshtein')

In [25]:
stop_words = {'pas', 'le', 'en', 'pour', 'ou', 'ce', 'de', 'dans', 'du', 'and', 'un', 'sur', 'et',
              'of', 'est', 'par', 'la', 'les', 'dont', 'au', 'des', 'que'}

In [27]:
param_grid = [{'Splitter__splitter_func': splitter_funcs,
               'SimilaritySelector__similarity': ['projection', 'cosine'],
               'SimilaritySelector__count_vect_kwargs': [{'stop_words': stop_words},
                                                         {'stop_words': None},
                                                        ],
              }
             ]
search = GridSearchCV(process_pipe,
                      param_grid,
                      cv=10, 
                      scoring= lev_scorer,
                      n_jobs=-1,
                      verbose=1,
                     ).fit(train, train['ingredients'])

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   10.3s


Launching 8 processes.


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   35.5s finished


In [28]:
search.cv_results_

{'mean_fit_time': array([0.5242799 , 0.29072883, 0.20813572, 0.19394908, 0.232284  ,
        0.20478857, 0.2006624 , 0.2214685 , 0.20743845, 0.20030537,
        0.2202004 , 0.20550847]),
 'std_fit_time': array([0.10931549, 0.07337048, 0.00735431, 0.00745012, 0.01088503,
        0.00518096, 0.00743987, 0.00959173, 0.00558991, 0.01060589,
        0.01137143, 0.00863639]),
 'mean_score_time': array([0.37875197, 0.2663589 , 0.19132805, 0.1705699 , 0.24689152,
        0.16716609, 0.18274245, 0.21833038, 0.18290744, 0.16947865,
        0.23152869, 0.16484597]),
 'std_score_time': array([0.06868515, 0.04970968, 0.02122265, 0.00945115, 0.01963565,
        0.01001417, 0.00642451, 0.01058498, 0.00990497, 0.01345436,
        0.0162835 , 0.00923047]),
 'param_SimilaritySelector__count_vect_kwargs': masked_array(data=[{'stop_words': {'en', 'un', 'of', 'pour', 'le', 'ce', 'du', 'dont', 'dans', 'par', 'les', 'et', 'au', 'de', 'la', 'que', 'des', 'sur', 'pas', 'and', 'ou', 'est'}},
                   

On voit que la fonction de split la plus efficace est la fonction qui applique la regex (deux retours chariots parmi des whitespaces).

In [40]:
labels = list(product(['Avec stopwords', 'Sans stopwords'],
                      ['Projection l2/l1', 'Cosinus'],
                      ['Split 1', 'Split 2', 'Split 3'], 
                 ))

for i in range(len(search.cv_results_['rank_test_score'])):
    str_result = f"{search.cv_results_['mean_test_score'][i]:.2%} +/- {search.cv_results_['std_test_score'][i]:.2%}"
    print(', '.join(labels[i]), str_result)

12
Avec stopwords, Projection l2/l1, Split 1 54.74% +/- 4.48%
Avec stopwords, Projection l2/l1, Split 2 41.52% +/- 3.89%
Avec stopwords, Projection l2/l1, Split 3 57.84% +/- 5.73%
Avec stopwords, Cosinus, Split 1 50.88% +/- 5.89%
Avec stopwords, Cosinus, Split 2 29.34% +/- 5.52%
Avec stopwords, Cosinus, Split 3 53.20% +/- 7.45%
Sans stopwords, Projection l2/l1, Split 1 49.81% +/- 5.99%
Sans stopwords, Projection l2/l1, Split 2 38.24% +/- 4.17%
Sans stopwords, Projection l2/l1, Split 3 52.56% +/- 6.82%
Sans stopwords, Cosinus, Split 1 41.03% +/- 4.46%
Sans stopwords, Cosinus, Split 2 26.26% +/- 2.40%
Sans stopwords, Cosinus, Split 3 42.71% +/- 4.99%
